# Textual Similarity Analysis Tool

## Overview
This notebook, authored by Jesus Mendez, is designed to perform detailed similarity analysis between text documents. It implements various representation techniques and similarity measures to assess the closeness of textual data.

## Functionality
- **Text Processing:** Utilizes custom functions to read and preprocess text data.
- **Vector Representations:** Converts text into binary, frequency, and TF-IDF vector representations.
- **Cosine Similarity Measurement:** Computes cosine similarity between a document and a corpus to find the most similar texts.
- **Results Output:** Outputs the similarity results into an Excel file, organizing data across multiple sheets for different document comparisons.

## Key Components
- **Binary Representation:** Considers the presence or absence of terms in documents.
- **Frequency Representation:** Takes into account the frequency of terms in the text.
- **TF-IDF Representation:** Reflects the importance of terms in documents relative to a corpus.
- **Cosine Similarity:** A method used to determine the cosine of the angle between two non-zero vectors in the space of their representations, indicative of their similarity.

## Output
- An Excel file named `tabla.xlsx` containing the similarity scores and the corresponding text excerpts, structured to provide clear insights for each type of vector representation.

## Author
- **Name:** Jesus Mendez

This tool is essential for researchers and analysts involved in natural language processing, allowing them to quantitatively compare texts and identify relevant patterns or similarities.


### Importing Libraries and Modules
This block imports necessary libraries for text processing and similarity analysis, including `numpy` for numerical operations, `sklearn` for feature extraction and similarity calculation functions, and custom modules for text normalization and manipulation.


In [1]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import normalizar
import openpyxl

### Functions to Retrieve Corpus and Documents
Defines functions to load and process texts from files. This includes reading a complete corpus or individual documents, with options to normalize the text using a custom module if required.


In [2]:
def getCorpus(fileName, norm = False):
    if norm:
            corpus = normalizar.generarCorpus(fileName)
            corpusProcesado = normalizar.procesarCorpus(corpus)
    else: 
        with open(fileName, 'r', encoding='utf-8') as f:
            corpusProcesado = f.readlines()
    return corpusProcesado
    
def getDocuments(fileName, norm = True):
    if norm:
        corpusProcesado = normalizar.procesarCadena(fileName)
    else: 
        with open(fileName, 'r', encoding='utf-8') as f:
            corpusProcesado = f.read()
    
    return corpusProcesado

### Text Representation
Implements three types of vector representations for texts: binary, term frequency, and TF-IDF. These functions prepare the corpus and documents for similarity calculation by transforming the text into numerical vectors based on the presence and frequency of words.


In [3]:
def repBinaria(corpus, doc):
    vectorizador_binario = CountVectorizer(binary=True, token_pattern=r'(?u)\w\w+|\w\w+\n|\.')
    vectorCorpus = vectorizador_binario.fit_transform(corpus)
    vectorDocumento = vectorizador_binario.transform([doc])
    
    return vectorCorpus, vectorDocumento 

def repFrecuencia(corpus, doc):
    vectorizador_frecuencia = CountVectorizer(token_pattern=r'(?u)\w\w+|\w\w+\n|\.')
    vectorCorpus = vectorizador_frecuencia.fit(corpus)
    vectorDocumento = vectorizador_frecuencia.transform([doc])
    
    return vectorCorpus, vectorDocumento 

def repTFIDF(corpus, doc):
    vectorizador_tfidf = TfidfVectorizer(token_pattern=r'(?u)\w\w+|\w\w+\n|\.')
    vectorCorpus = vectorizador_tfidf.fit(corpus)
    vectorDocumento = vectorizador_tfidf.transform([doc])
    
    return vectorCorpus, vectorDocumento 


### Cosine Similarity Calculation
Defines a function to calculate the cosine similarity between a document and a corpus, using the vector representations previously defined. This function is crucial for identifying the most similar documents within the corpus.


In [4]:
def similitudCoseno(corpus, doc, n = 10):
    similitud = cosine_similarity(doc, corpus)
    line_index = 0 #linea especifica del corpus
    similarities = similitud[line_index]
    index = np.argsort(similarities)[-n:]
    index = np.flipud(index)

    simValue = list()

    for i in index:
        simValue.append(cosine_similarity(doc, corpus[i]))

    return index, simValue

### Creating a Results Table in Excel
Uses a loop to process various documents, compare them with the corpus using different representations, and save the results in an Excel spreadsheet. Each document generates a separate sheet with the most similar texts and their corresponding similarity values.


In [9]:
def makeTable():

    corpus = getCorpus("corpus_procesado.txt")

    docs = ['prueba 1 (medicamentos)', 'prueba 2 (horario de verano)', 'prueba 3 (pemex)', 'prueba 4 (IPN)', 'prueba 5 (vaticano)']

    libro = openpyxl.Workbook()

    for i in docs:

        doc = getDocuments("./pruebas/"+i+'.txt')

        nueva_hoja = libro.create_sheet(i)
        libro.active = nueva_hoja

        nueva_hoja.cell(row=1, column=1, value=i)
        nueva_hoja.cell(row=1, column=2, value="Contenido")
        nueva_hoja.cell(row=1, column=3, value="Valor Similitud")

        for i in range(3):
            if i == 0:
                vectorCorpus, vectorDocumento = repBinaria(corpus, doc)
                array, distancia = similitudCoseno(vectorCorpus, vectorDocumento)
                for i in range(10):
                    nueva_hoja.cell(row=i+2, column=1, value="Representacion Binaria")
                    nueva_hoja.cell(row=i+2, column=2, value=corpus[array[i]])
                    nueva_hoja.cell(row=i+2, column=3, value=distancia[i][0][0])
            elif i == 1:
                vectorCorpus, vectorDocumento = repFrecuencia(corpus, doc)
                array, distancia = similitudCoseno(vectorCorpus, vectorDocumento)
                for i in range(10):
                    nueva_hoja.cell(row=i+13, column=1, value="Representación de frecuencias")
                    nueva_hoja.cell(row=i+13, column=2, value=corpus[array[i]])
                    nueva_hoja.cell(row=i+13, column=3, value=distancia[i][0][0])
            else: 
                vectorCorpus, vectorDocumento = repTFIDF(corpus, doc)
                array, distancia = similitudCoseno(vectorCorpus, vectorDocumento)
                for i in range(10):
                    nueva_hoja.cell(row=i+24, column=1, value="TD-IDF")
                    nueva_hoja.cell(row=i+24, column=2, value=corpus[array[i]])
                    nueva_hoja.cell(row=i+24, column=3, value=distancia[i][0][0])

    libro.save("tabla.xlsx")


### Example of Binary Representation and Similarity Usage
Applies the binary representation and cosine similarity calculation to a test document and the corpus, showing examples of similar texts found.


In [6]:
corpus = getCorpus("corpus_procesado.txt")
doc = getDocuments('./pruebas/prueba 1 (medicamentos).txt')

print(doc)


ciudad MÉXICO . problema sector salud Veracruz poder estar vigente , ir más allá quimioterapia medicamento apócrifo , aplicado supuestamente niño cáncer administración gobernador Fidel Herrera Javier Duarte . José Narro Robles , secretario Salud , revelar martes brigada 12 especialista Cofepris personal Secretaría Salud descubrir 11 tonelada medicamento caduco 47 mil prueba vih registro sanitario . ¿ aquí haber problema ? sí , haber problema , decir claridad , haber detectar problema . gustar no presentar haber presentar , tener corregir situación ” , decir .   durante después reunión plenario Partido Verde Ecologista , Senado República , funcionario contestar tema central investigación , quimioterapia apócrifa niño cáncer , estar investigar registro 2010 2013 . haber encontrar ser evidencia documental caso registrar año 2010 conclusión avanzar año 2011 . no tener evidencia momento afectación , niño paciente , tener estudiar totalidad expediente clínico poder llegar conclusión ” , deci

In [7]:

c, d = repBinaria(corpus, doc)
arr, dis = similitudCoseno(c,d)

In [8]:
for i in arr:
    print(corpus[i])

ciudad MÉXICO . problema sector salud Veracruz poder estar vigente , ir más allá quimioterapia medicamento apócrifo , aplicado supuestamente niño cáncer administración gobernador Fidel Herrera Javier Duarte . José Narro Robles , secretario Salud , revelar martes brigada 12 especialista Cofepris personal Secretaría Salud descubrir 11 tonelada medicamento caduco 47 mil prueba vih registro sanitario . ¿ aquí haber problema ? sí , haber problema , decir claridad , haber detectar problema . gustar no presentar haber presentar , tener corregir situación ” , decir .   durante después reunión plenario Partido Verde Ecologista , Senado República , funcionario contestar tema central investigación , quimioterapia apócrifa niño cáncer , estar investigar registro 2010 2013 . haber encontrar ser evidencia documental caso registrar año 2010 conclusión avanzar año 2011 . no tener evidencia momento afectación , niño paciente , tener estudiar totalidad expediente clínico poder llegar conclusión ” , deci